In [8]:
import pandas as pd
import numpy as np

In [129]:
ibov_historic = pd.read_csv("./database.csv")

In [130]:
ibov_historic.head()

,date,high,volume,close,low,open,adjclose,rsi,BOLM,BOLU,BOLD,STOCH,%R,ROC,SMA,EMA,MACD,CCI,OBV,STOCHrsi
0,2016-12-01,61911.0,5352800.0,59507.0,59058.0,61906.0,59507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59507.000000,0.000000,NaN,0.0,NaN
1,2016-12-02,60379.0,4344200.0,60316.0,58092.0,59499.0,60316.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59940.392857,-18.150641,NaN,4344200.0,NaN
2,2016-12-05,60720.0,2861200.0,59832.0,59635.0,60322.0,59832.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59898.986418,-8.118708,NaN,1483000.0,NaN
3,2016-12-06,61235.0,3606500.0,61088.0,59396.0,59828.0,61088.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60262.739213,-48.704421,NaN,5089500.0,NaN
4,2016-12-07,61918.0,3841700.0,61414.0,61062.0,61111.0,61414.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60563.101391,-82.705030,NaN,8931200.0,NaN


# EDA

In [5]:
from pandas_profiling import ProfileReport

In [6]:
profile = ProfileReport(ibov_historic, title="Profiling Report IBOV", explorative=True)

In [7]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/33 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [131]:
base_modelagem = ibov_historic.copy()

In [132]:
base_modelagem.head()

,date,high,volume,close,low,open,adjclose,rsi,BOLM,BOLU,BOLD,STOCH,%R,ROC,SMA,EMA,MACD,CCI,OBV,STOCHrsi
0,2016-12-01,61911.0,5352800.0,59507.0,59058.0,61906.0,59507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59507.000000,0.000000,NaN,0.0,NaN
1,2016-12-02,60379.0,4344200.0,60316.0,58092.0,59499.0,60316.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59940.392857,-18.150641,NaN,4344200.0,NaN
2,2016-12-05,60720.0,2861200.0,59832.0,59635.0,60322.0,59832.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59898.986418,-8.118708,NaN,1483000.0,NaN
3,2016-12-06,61235.0,3606500.0,61088.0,59396.0,59828.0,61088.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60262.739213,-48.704421,NaN,5089500.0,NaN
4,2016-12-07,61918.0,3841700.0,61414.0,61062.0,61111.0,61414.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60563.101391,-82.705030,NaN,8931200.0,NaN


In [133]:
base_modelagem.shape

(1233, 20)

In [135]:
base_modelagem = base_modelagem.set_index("date")

### Função para criar os targets

In [136]:
def target_build(close, delta):
    """Função para construir o alvo da modelagem, indicando se
    houve uma variação positiva ou negativa, em um intervalo delta"""
    
    x_arr = np.array(close.copy())
    y_arr = np.array(close.copy())
    
    target = np.array([])
    
    for i in range(0, (len(x_arr) - delta)):
        
        variation = y_arr[i+delta] - x_arr[i]
        
        #import pdb; pdb.set_trace()
        
        if variation > 0:
            target = np.append(target, 1)
        elif variation < 0:
            target = np.append(target, -1)
        else:
            target = np.append(target, 0)
            
    for k in range(0, delta):
        target = np.append(target, np.nan)
        
    return target
            

In [137]:
# Aplicando a função para criar o alvo de para prever variação positiva, negativa ou sem variação de 1 a 5 dias
for i in range(1, 6):
    name = "target_" + str(i) + "_dias"
    base_modelagem[name] = target_build(base_modelagem.close, i)

In [138]:
base_modelagem.head()

,high,volume,close,low,open,adjclose,rsi,BOLM,BOLU,BOLD,...,EMA,MACD,CCI,OBV,STOCHrsi,target_1_dias,target_2_dias,target_3_dias,target_4_dias,target_5_dias
date,,,,,,,,,,,,,,,,,,,,,
2016-12-01,61911.0,5352800.0,59507.0,59058.0,61906.0,59507.0,NaN,NaN,NaN,NaN,...,59507.000000,0.000000,NaN,0.0,NaN,1.0,1.0,1.0,1.0,1.0
2016-12-02,60379.0,4344200.0,60316.0,58092.0,59499.0,60316.0,NaN,NaN,NaN,NaN,...,59940.392857,-18.150641,NaN,4344200.0,NaN,-1.0,1.0,1.0,1.0,1.0
2016-12-05,60720.0,2861200.0,59832.0,59635.0,60322.0,59832.0,NaN,NaN,NaN,NaN,...,59898.986418,-8.118708,NaN,1483000.0,NaN,1.0,1.0,1.0,1.0,-1.0
2016-12-06,61235.0,3606500.0,61088.0,59396.0,59828.0,61088.0,NaN,NaN,NaN,NaN,...,60262.739213,-48.704421,NaN,5089500.0,NaN,1.0,-1.0,-1.0,-1.0,-1.0
2016-12-07,61918.0,3841700.0,61414.0,61062.0,61111.0,61414.0,NaN,NaN,NaN,NaN,...,60563.101391,-82.705030,NaN,8931200.0,NaN,-1.0,-1.0,-1.0,-1.0,-1.0


### Distribuição dos targets

#### 1 dia

In [139]:
base_modelagem.target_1_dias.value_counts(dropna=False)

 1.0    658
-1.0    573
 0.0      1
 NaN      1
Name: target_1_dias, dtype: int64

#### 2 dias

In [140]:
base_modelagem.target_2_dias.value_counts(dropna=False)

 1.0    679
-1.0    552
 NaN      2
Name: target_2_dias, dtype: int64

#### 3 dias

In [141]:
base_modelagem.target_3_dias.value_counts(dropna=False)

 1.0    696
-1.0    533
 NaN      3
 0.0      1
Name: target_3_dias, dtype: int64

#### 4 dias

In [142]:
base_modelagem.target_4_dias.value_counts(dropna=False)

 1.0    703
-1.0    526
 NaN      4
Name: target_4_dias, dtype: int64

#### 5 dias

In [143]:
base_modelagem.target_5_dias.value_counts(dropna=False)

 1.0    706
-1.0    522
 NaN      5
Name: target_5_dias, dtype: int64

# Feature Selection

In [175]:
base_1_dia = base_modelagem.drop(columns=["high","volume","close","low","open","adjclose","target_2_dias","target_3_dias","target_4_dias","target_5_dias"])

In [176]:
base_1_dia.head()

,rsi,BOLM,BOLU,BOLD,STOCH,%R,ROC,SMA,EMA,MACD,CCI,OBV,STOCHrsi,target_1_dias
date,,,,,,,,,,,,,,
2016-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59507.000000,0.000000,NaN,0.0,NaN,1.0
2016-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59940.392857,-18.150641,NaN,4344200.0,NaN,-1.0
2016-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59898.986418,-8.118708,NaN,1483000.0,NaN,1.0
2016-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60262.739213,-48.704421,NaN,5089500.0,NaN,1.0
2016-12-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60563.101391,-82.705030,NaN,8931200.0,NaN,-1.0


### Removendo valores faltantes

In [177]:
base_1_dia.dropna(inplace=True)

In [178]:
base_1_dia.shape

(1213, 14)

In [179]:
base_1_dia.corr()

,rsi,BOLM,BOLU,BOLD,STOCH,%R,ROC,SMA,EMA,MACD,CCI,OBV,STOCHrsi,target_1_dias
rsi,1.000000,-0.145191,-0.147481,-0.138321,0.865862,0.824878,0.832214,-0.101702,-0.089346,-0.603028,0.852298,-0.062168,1.000000,-0.008576
BOLM,-0.145191,1.000000,0.985230,0.984232,-0.107636,-0.111582,-0.133609,0.997741,0.997158,-0.036665,-0.175340,0.916875,-0.145191,-0.037620
BOLU,-0.147481,0.985230,1.000000,0.939407,-0.113186,-0.115568,-0.195728,0.977352,0.976729,0.057795,-0.179868,0.914856,-0.147481,-0.035806
BOLD,-0.138321,0.984232,0.939407,1.000000,-0.098563,-0.104003,-0.065297,0.987847,0.987323,-0.133099,-0.165223,0.890515,-0.138321,-0.038327
STOCH,0.865862,-0.107636,-0.113186,-0.098563,1.000000,0.910598,0.738849,-0.072471,-0.055308,-0.527845,0.892921,-0.019802,0.865862,-0.028552
%R,0.824878,-0.111582,-0.115568,-0.104003,0.910598,1.000000,0.704391,-0.082329,-0.065723,-0.449915,0.947403,-0.017503,0.824878,-0.067695
ROC,0.832214,-0.133609,-0.195728,-0.065297,0.738849,0.704391,1.000000,-0.078167,-0.063024,-0.750925,0.727794,-0.028267,0.832214,-0.001920
SMA,-0.101702,0.997741,0.977352,0.987847,-0.072471,-0.082329,-0.078167,1.000000,0.999446,-0.095976,-0.142047,0.916768,-0.101702,-0.036421
EMA,-0.089346,0.997158,0.976729,0.987323,-0.055308,-0.065723,-0.063024,0.999446,1.000000,-0.097479,-0.127193,0.920399,-0.089346,-0.036912
MACD,-0.603028,-0.036665,0.057795,-0.133099,-0.527845,-0.449915,-0.750925,-0.095976,-0.097479,1.000000,-0.514698,-0.015902,-0.603028,-0.003814


In [194]:
corr = base_1_dia.corr()

### Seleção das colunas com base na correlação

In [195]:
columns = np.full((len(corr),), True, dtype=bool)
for i in range(len(corr)):
    for j in range(i+1, len(corr)):
        if corr.iloc[i,j] >= 0.9 or corr.iloc[i,j] <= -0.9:
            if columns[j]:
                columns[j] = False

In [196]:
selected_columns = base_1_dia.columns[columns]

base_1_dia = base_1_dia[selected_columns]

In [197]:
base_1_dia

,rsi,BOLM,STOCH,ROC,MACD,target_1_dias
date,,,,,,
2016-12-28,44.106414,59116.700000,46.773726,-1.475023,169.263987,1.0
2016-12-29,48.257441,59104.650000,65.796108,-0.452885,45.597004,-1.0
2017-01-02,52.855949,59111.316667,80.512145,0.692813,-7.895930,1.0
2017-01-03,63.616815,59162.533333,90.947321,4.272870,-200.450604,-1.0
2017-01-04,69.965709,59214.400000,91.279254,5.801209,-332.169545,1.0
...,...,...,...,...,...,...
2021-11-19,41.975234,105546.316667,9.815091,-2.525898,1913.318737,-1.0
2021-11-22,45.506680,105290.333333,8.209366,-1.332354,1982.762595,1.0
2021-11-23,43.637099,105160.200000,14.919765,-1.788709,1891.645816,1.0


## Modelagem

### Divisão out of time da base

In [206]:
base_1_dia = base_1_dia[base_1_dia.target_1_dias != 0]

,rsi,BOLM,STOCH,ROC,MACD,target_1_dias
date,,,,,,
2017-12-28,75.909993,73448.966667,97.733064,5.400968,-554.832852,0.0


In [198]:
X = base_1_dia.drop(columns=["target_1_dias"])
y = base_1_dia["target_1_dias"]

In [199]:
X_train = X[:int(X.shape[0]*0.7)]
X_test = X[int(X.shape[0]*0.7):]
y_train = y[:int(X.shape[0]*0.7)]
y_test = y[int(X.shape[0]*0.7):]

In [200]:
from sklearn.ensemble import RandomForestClassifier

In [201]:
clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, random_state=0)

In [202]:
y_pred = clf.predict(X_test)

In [203]:
from sklearn.metrics import confusion_matrix, classification_report

In [204]:
print(confusion_matrix(y_test, y_pred, normalize='true'))

[[0.5872093  0.4127907 ]
 [0.58854167 0.41145833]]


In [205]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.47      0.59      0.52       172
         1.0       0.53      0.41      0.46       192

    accuracy                           0.49       364
   macro avg       0.50      0.50      0.49       364
weighted avg       0.50      0.49      0.49       364

